<a href="https://colab.research.google.com/github/annisaizzatullatifa/Goodplant-Massive-Challenge/blob/main/Coba_TA_metadata_steam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics.pairwise import cosine_similarity

# Mount Google Drive
drive.mount('/content/drive')

# Path ke file dataset di Google Drive (ubah sesuai lokasi penyimpanan)
file_path = "/content/drive/My Drive/datasetTA/datasetFinal/cleaned_steam_metadata.csv"

# Load dataset
df = pd.read_csv(file_path)

# Cek data
print(df.head())


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
   appid                       name         developer publisher  \
0     10             Counter-Strike             Valve     Valve   
1     20      Team Fortress Classic             Valve     Valve   
2     30              Day of Defeat             Valve     Valve   
3     40         Deathmatch Classic             Valve     Valve   
4     50  Half-Life: Opposing Force  Gearbox Software     Valve   

                                          categories  genres  
0  Multi-player;Online Multi-Player;Local Multi-P...  Action  
1  Multi-player;Online Multi-Player;Local Multi-P...  Action  
2              Multi-player;Valve Anti-Cheat enabled  Action  
3  Multi-player;Online Multi-Player;Local Multi-P...  Action  
4  Single-player;Multi-player;Valve Anti-Cheat en...  Action  


In [31]:
# Isi nilai NaN
df["developer"].fillna("Unknown", inplace=True)
df["publisher"].fillna("Unknown", inplace=True)
df["categories"].fillna("None", inplace=True)
df["genres"].fillna("None", inplace=True)

# One-Hot Encoding developer & publisher
encoder = OneHotEncoder(sparse_output=False, handle_unknown="ignore")
developer_ohe = encoder.fit_transform(df[["developer"]])
developer_df = pd.DataFrame(developer_ohe, columns=encoder.get_feature_names_out(["developer"]), index=df.index)

encoder_publisher = OneHotEncoder(sparse_output=False, handle_unknown="ignore")
publisher_ohe = encoder_publisher.fit_transform(df[["publisher"]])
publisher_df = pd.DataFrame(publisher_ohe, columns=encoder_publisher.get_feature_names_out(["publisher"]), index=df.index)

# CountVectorizer untuk categories & genres
vectorizer_categories = CountVectorizer()
categories_bow = vectorizer_categories.fit_transform(df["categories"]).toarray()
categories_df = pd.DataFrame(categories_bow, columns=vectorizer_categories.get_feature_names_out(), index=df.index)

vectorizer_genres = CountVectorizer()
genres_bow = vectorizer_genres.fit_transform(df["genres"]).toarray()
genres_df = pd.DataFrame(genres_bow, columns=vectorizer_genres.get_feature_names_out(), index=df.index)

# Gabungkan semua fitur menjadi satu DataFrame
final_features = pd.concat([developer_df, publisher_df, categories_df, genres_df], axis=1)

print(f"Shape of feature matrix: {final_features.shape}")  # Harus sesuai dengan df.shape[0]


<ipython-input-31-57b6f66d24bf>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["developer"].fillna("Unknown", inplace=True)
<ipython-input-31-57b6f66d24bf>:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using

Shape of feature matrix: (5298, 6143)


In [32]:
missing_games = set(df.index) - set(final_features.index)
print("Jumlah game yang hilang:", len(missing_games))
print(missing_games)


Jumlah game yang hilang: 0
set()


In [33]:
# Hitung jumlah fitur dari setiap encoding
num_dev_features = developer_df.shape[1]  # Jumlah kolom dari One-Hot Encoding developer
num_pub_features = publisher_df.shape[1]  # Jumlah kolom dari One-Hot Encoding publisher
num_cat_features = categories_df.shape[1]  # Jumlah kolom dari Bag of Words categories
num_gen_features = genres_df.shape[1]  # Jumlah kolom dari Bag of Words genres

# Hitung total fitur yang diharapkan
total_expected_features = num_dev_features + num_pub_features + num_cat_features + num_gen_features

# Cetak hasilnya
print(f"Developer features: {num_dev_features}")
print(f"Publisher features: {num_pub_features}")
print(f"Categories features: {num_cat_features}")
print(f"Genres features: {num_gen_features}")
print(f"Total expected features: {total_expected_features}")

# Pastikan sesuai dengan final_features.shape
assert final_features.shape[1] == total_expected_features, "Jumlah fitur tidak sesuai!"
print(f"✅ Shape final_features: {final_features.shape}, sesuai dengan perhitungan.")


Developer features: 3518
Publisher features: 2540
Categories features: 47
Genres features: 38
Total expected features: 6143
✅ Shape final_features: (5298, 6143), sesuai dengan perhitungan.


In [34]:
print(final_features.isna().sum().sum())  # Harusnya hasilnya 0


0


In [35]:
print(final_features.describe())  # Untuk melihat range nilai tiap fitur


       developer_07th Expansion  developer_100 Stones Interactive  \
count               5298.000000                       5298.000000   
mean                   0.000944                          0.000189   
std                    0.030709                          0.013739   
min                    0.000000                          0.000000   
25%                    0.000000                          0.000000   
50%                    0.000000                          0.000000   
75%                    0.000000                          0.000000   
max                    1.000000                          1.000000   

       developer_10Ants Hill  developer_10th Art Studio  developer_10tons Ltd  \
count            5298.000000                5298.000000           5298.000000   
mean                0.000189                   0.000189              0.000378   
std                 0.013739                   0.013739              0.019428   
min                 0.000000                   0.00000

In [36]:
sparsity = 1.0 - (final_features.astype(bool).sum().sum() / final_features.size)
print(f"Sparsity: {sparsity:.4f}")  # Jika terlalu sparse (> 90%), pertimbangkan reduksi dimensi


Sparsity: 0.9979


In [38]:
# Simpan hasil rekomendasi sebelum perubahan fitur
previous_recommendations = model.predict(previous_features)

# Jalankan model dengan fitur baru
new_recommendations = model.predict(final_features)

# Pastikan model tidak error dan output sesuai format
assert len(new_recommendations) == len(previous_recommendations), "Jumlah rekomendasi berubah!"


NameError: name 'model' is not defined

In [22]:
# 1. Mengisi nilai kosong dengan "Unknown"
df["publisher"].fillna("Unknown", inplace=True)
df["developer"].fillna("Unknown", inplace=True)

# 2. Menghapus karakter whitespace atau tab yang tidak perlu
df["publisher"] = df["publisher"].str.strip()
df["developer"] = df["developer"].str.strip()

# 3. Memisahkan nilai yang memiliki beberapa developer/publisher
df = df.assign(developer=df["developer"].str.split(";")).explode("developer")
df = df.assign(publisher=df["publisher"].str.split(";")).explode("publisher")

# 4. Memastikan semua string terbaca dengan baik
df["developer"] = df["developer"].astype(str)
df["publisher"] = df["publisher"].astype(str)


<ipython-input-22-7de16a6927d1>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["publisher"].fillna("Unknown", inplace=True)
<ipython-input-22-7de16a6927d1>:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using

In [24]:
print(f"Ukuran df: {df.shape}")  # Berapa total game dalam df
print(f"Ukuran final_features: {final_features.shape}")  # Berapa banyak game yang masuk ke cosine similarity
print(f"Ukuran cosine_sim: {cosine_sim.shape}")  # Harus (N x N) dengan N sesuai jumlah game yang diproses


Ukuran df: (6378, 6)
Ukuran final_features: (5299, 6398)
Ukuran cosine_sim: (5299, 5299)


In [25]:
print("Game dalam df tetapi tidak dalam final_features:")
print(set(df["name"]) - set(df.iloc[:final_features.shape[0]]["name"]))


Game dalam df tetapi tidak dalam final_features:
{'Mini Thief', 'The Brookhaven Experiment', 'Zombo Buster Rising', 'Settled', 'SkyTime', 'The Housewife', "神明的一天世界(God's One Day World)", 'Darconika: The Cube of Soul', 'BLUE REVOLVER', 'Frog Climbers', 'Big Bang Empire', 'Zombie Defense', 'iGrow Game', 'Starr Mazer: DSP', 'ORCS', 'Experience', 'Iron Impact', 'Demolish & Build 2017', 'Existentia', 'Pub Encounter', 'The Body VR: Journey Inside a Cell', 'Tricky Towers', 'ROD: Revolt Of Defense', 'Table Top Racing: World Tour', 'Shot Shot Tactic', 'Airport Madness 3D', 'Final Quest II', 'HACK_IT', 'Blameless', 'Squeezone', 'We Know the Devil', 'What The Box?', 'Eleven: Table Tennis VR', 'Job Simulator', 'Woodle Tree 2: Worlds', 'Sparkle ZERO', 'AdVenture Communist', 'The Narrator Is a DICK', 'Invisible Mind', 'Gone In November', 'Island 359™', 'Star Drifter', 'Zombillie', 'Senza Peso', 'Pirate Pop Plus', 'Age of Cavemen', 'Atonement 2: Ruptured by Despair', 'Firebird - La Peri', "Alice's Pa

In [26]:
game_names = df.iloc[:final_features.shape[0]]["name"].values  # Ambil hanya sebanyak final_features
cosine_sim_df = pd.DataFrame(cosine_sim, index=game_names, columns=game_names)
print(cosine_sim_df.head())


                           Counter-Strike  Team Fortress Classic  \
Counter-Strike                   1.000000               1.000000   
Team Fortress Classic            1.000000               1.000000   
Day of Defeat                    0.833950               0.833950   
Deathmatch Classic               1.000000               1.000000   
Half-Life: Opposing Force        0.800641               0.800641   

                           Day of Defeat  Deathmatch Classic  \
Counter-Strike                   0.83395            1.000000   
Team Fortress Classic            0.83395            1.000000   
Day of Defeat                    1.00000            0.833950   
Deathmatch Classic               0.83395            1.000000   
Half-Life: Opposing Force        0.83205            0.800641   

                           Half-Life: Opposing Force  Ricochet  Half-Life  \
Counter-Strike                              0.800641  0.962250   0.903525   
Team Fortress Classic                       0.800641

In [28]:
print(df["name"].duplicated().sum())  # Jika > 0, ada duplikasi


1080


In [29]:
df = df.drop_duplicates(subset="name").reset_index(drop=True)


In [30]:
# Menghitung Cosine Similarity
cosine_sim = cosine_similarity(final_features)

# Konversi ke DataFrame untuk memudahkan analisis
cosine_sim_df = pd.DataFrame(cosine_sim, index=df["name"], columns=df["name"])

print(cosine_sim_df.head())  # Cek beberapa nilai similarity


ValueError: Shape of passed values is (5299, 5299), indices imply (5298, 5298)

In [21]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
developer_df = pd.DataFrame(scaler.fit_transform(developer_df), columns=developer_df.columns)
publisher_df = pd.DataFrame(scaler.fit_transform(publisher_df), columns=publisher_df.columns)
categories_df = pd.DataFrame(scaler.fit_transform(categories_df), columns=categories_df.columns)
genres_df = pd.DataFrame(scaler.fit_transform(genres_df), columns=genres_df.columns)

# Contoh bobot (atur sesuai kebutuhan)
weights = {
    "developer": 1.0,
    "publisher": 0.5,
    "categories": 1.5,
    "genres": 2.0
}

# Pastikan tidak ada NaN dalam setiap DataFrame sebelum pembobotan
developer_df = developer_df.fillna(0)
publisher_df = publisher_df.fillna(0)
categories_df = categories_df.fillna(0)
genres_df = genres_df.fillna(0)

# Terapkan bobot ke vektor fitur
weighted_features = (
    developer_df * weights["developer"] +
    publisher_df * weights["publisher"] +
    categories_df * weights["categories"] +
    genres_df * weights["genres"]
)

# Pastikan weighted_features tidak memiliki NaN
weighted_features = weighted_features.fillna(0)

# Hitung ulang Cosine Similarity setelah pembobotan
weighted_cosine_sim = cosine_similarity(weighted_features)

# Konversi ke DataFrame
weighted_cosine_sim_df = pd.DataFrame(weighted_cosine_sim, index=df["name"], columns=df["name"])

print(weighted_cosine_sim_df.head())  # Cek hasil similarity berbobot

print(developer_df.nunique())  # Lihat jumlah nilai unik per kolom
print(publisher_df.nunique())
print(categories_df.nunique())
print(genres_df.nunique())

name                       Counter-Strike  Team Fortress Classic  \
name                                                               
Counter-Strike                        0.0                    0.0   
Team Fortress Classic                 0.0                    0.0   
Day of Defeat                         0.0                    0.0   
Deathmatch Classic                    0.0                    0.0   
Half-Life: Opposing Force             0.0                    0.0   

name                       Day of Defeat  Deathmatch Classic  \
name                                                           
Counter-Strike                       0.0                 0.0   
Team Fortress Classic                0.0                 0.0   
Day of Defeat                        0.0                 0.0   
Deathmatch Classic                   0.0                 0.0   
Half-Life: Opposing Force            0.0                 0.0   

name                       Half-Life: Opposing Force  Ricochet  Half-Life 